# 如何并行调用可运行组件 How to invoke runnables in parallel

RunnableParallel 原语本质上是一个字典，其值是可运行对象（或可以强制转换为可运行对象的东西，如函数）。  
它并行运行其所有值，并且每个值都使用 RunnableParallel 的整体输入进行调用。  
最终返回值是一个字典，其每个值的结果都位于其相应的键下。

## 使用 RunnableParallels 进行格式化
RunnableParallels 对于并行化操作很有用，但也可用于操纵一个 Runnable 的输出以匹配序列中下一个 Runnable 的输入格式。  
您可以使用它们来拆分或分叉链，以便多个组件可以并行处理输入。  
稍后，其他组件可以连接或合并结果以合成最终响应。这种类型的链会创建一个如下所示的计算图：

以下内容中，传递给`prompt`的输入预期是一个包含"context"和"question"两个键的映射（map）。  
用户输入仅仅是问题部分。所以我们需要用我们的检索器获取上下文，并将用户的输入作为"question"键下的内容传递进去。

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])
model = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4"
)

vectorstore = FAISS.from_texts(["张三在食为天工作"],embedding=embeddings)
retriever = vectorstore.as_retriever()
template = """请仅根据以下上下文回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

retrieval_chain = (
    {"context":retriever,"question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("张三在哪工作?")

'根据文档内容，张三在食为天工作。'

请注意，当将 RunnableParallel 与另一个 Runnable 组合时，我们甚至不需要将字典包装在 RunnableParallel 类中 ,类型转换已经为我们处理好了。  
在链式结构的上下文中，以下两种方式是等效的：

```python
{"context": retriever, "question": RunnablePassthrough()}
```

```python
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
```

```python
RunnableParallel(context=retriever, question=RunnablePassthrough())
```

```python
from langchain_core.runnables import RunnableParallel
RunnableParallel({"context": retriever, "question": RunnablePassthrough()}).invoke("张三在哪工作?")
RunnableParallel(context=retriever, question=RunnablePassthrough()).invoke("张三在哪工作?")
```

## 使用itemgetter作为简写
请注意，当与RunnableParallel结合使用时，您可以使用Python的itemgetter作为从映射中提取数据的简写方式。  
有关itemgetter的更多信息，您可以在Python文档中找到。

在下面的例子中，我们使用itemgetter从映射中提取特定的键：

In [11]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

model = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4"
)


vectorestore = FAISS.from_texts(["张三喜欢是买辣鸡腿堡"],embedding=embeddings)
retriever = vectorestore.as_retriever()

template = """仅根据以下上下文回答问题：
{context}

问题：{question}

使用以下语言回答：{language}
"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
chain = (
    {
        "context":itemgetter("question") | retriever,
        "question":itemgetter("question"),
        "language":itemgetter("language")
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question":"张三的口味是什么？","language":"日文"})

'張三の味覚は辛い鶏のレギントンが好きです。'

## 并行化步骤
RunnableParallels使得同时执行多个Runnable变得简单，并且可以将这些Runnable的输出作为一个映射（map）返回。

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4"
)

joke_chain = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话") | model
poem_chain = (ChatPromptTemplate.from_template("写一首关于{topic}的两行诗") | model)

map_chain = RunnableParallel(joke=joke_chain,poem=poem_chain)
map_chain.invoke({"topic":"海燕"})

{'joke': AIMessage(content='好的，这是一个关于海燕的笑话：\n\n为什么海燕不会飞？\n因为它已经飞得太高，找不到可以落脚的地方了。', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 9, 'total_tokens': 38}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1120a472-5353-44da-bd7f-827c6eb97c6b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 29, 'total_tokens': 38}),
 'poem': AIMessage(content='海燕翱翔碧海间，\n勇敢无畏舞长天。', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 9, 'total_tokens': 22}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56c58ea2-a651-49c0-9589-5d2244a6eb48-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22})}

## 并行性
RunnableParallel 还可用于并行运行独立进程，因为映射中的每个 Runnable 都是并行执行的。  
例如，我们可以看到我们之前的 joke_chain、poem_chain 和 map_chain 都具有大致相同的运行时间，尽管 map_chain 执行了其他两个。

### %%timeit

%%timeit 是一个 Jupyter Notebook 或 IPython 环境中的魔术命令（magic command），用于测量代码单元格执行的时间。它会多次运行指定的代码块（默认情况下是数千次），然后给出平均执行时间、最慢执行时间和最快执行时间等统计信息，以此来提供代码执行效率的概览。

In [17]:
%%timeit

joke_chain.invoke({"topic": "狗"})

The slowest run took 4.79 times longer than the fastest. This could mean that an intermediate result is being cached.
2.28 s ± 1.63 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit

poem_chain.invoke({"topic": "狗"})

1.07 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit

map_chain.invoke({"topic": "狗"})

The slowest run took 5.76 times longer than the fastest. This could mean that an intermediate result is being cached.
3.07 s ± 2.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
